In [25]:
import pandas as pd

In [26]:
df= pd.read_csv('data/cleaned.csv')

In [27]:
df['type'].value_counts()

type
I    80677
E    25390
Name: count, dtype: int64

In [28]:
df.shape

(106067, 2)

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],   # Test different numbers of neighbors
    'weights': ['uniform', 'distance'], # Try different weighting methods
    'metric': ['euclidean', 'manhattan', 'minkowski'] # Try different distance metrics
}

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings("ignore")

# Load dataset
cleaned_data = pd.read_csv('data/cleaned.csv')

# Drop missing values in important columns
cleaned_data.dropna(subset=['posts', 'type'], inplace=True)

cleaned_data=cleaned_data.head(300)


# Convert MBTI types to binary classification (assuming only 'I' and 'E' exist)
cleaned_data['type'] = cleaned_data['type'].map({'I': 0, 'E': 1})

# Split data
X_train, X_test, y_train, y_test = train_test_split(cleaned_data['posts'], cleaned_data['type'], test_size=0.5, random_state=42)

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(stop_words='english', max_features=2000, ngram_range=(3, 3))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf=vectorizer.transform(X_test)

#apply SMOTE on training data
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
X_train_balanced, y_train_balanced = undersampler.fit_resample(X_train_tfidf, y_train)


In [32]:
# Perform Grid Search
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_balanced, y_train_balanced)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': [3, 5, 7, 9, 11],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [34]:
# Get best parameters
best_knn = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-validation Accuracy: {grid_search.best_score_}")

Best Parameters: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}
Best Cross-validation Accuracy: 0.7578947368421052
